In [ ]:
pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pa
import numpy as np

In [ ]:
train=pa.read_csv('train_data.csv')
test=pa.read_csv('test_data.csv')

In [ ]:
train.head()

,lable,text
0,1,ounc feather bowl hummingbird opec moment alab...
1,1,wulvob get medirc onlin qnb ikud viagra escape...
2,0,comput connect cnn com wednesday escapenumb ma...
3,1,univers degre obtain prosper futur money earn ...
4,0,thank answer guy know check rsync manual would...


In [ ]:
train=train.dropna()
test=test.dropna()
len(train)

62349

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenizer(train["text"][0])

{'input_ids': [101, 15068, 12273, 15550, 4605, 20364, 9001, 6728, 8586, 2617, 21862, 22083, 2102, 11748, 4801, 3089, 1040, 25152, 7852, 13109, 8684, 4078, 4842, 24264, 14905, 2594, 2018, 4948, 2002, 6199, 10861, 3363, 10930, 5603, 19585, 25277, 18900, 27345, 2044, 9581, 2290, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [ ]:
train["text"][4373]

'tue escapenumb may escapenumb hadley wickham wrote escapenumb escapenumb escapenumb rolf turner wrote escapenumb escapenumb escapenumb escapenumb escapenumb hadley wickham wrote two possibl solut display escapenumb escapenumb escapenumb escapenumb escapenumb r xescapenumb work without use xterm instal packag carbonel librari carbonel quartz clear life determin frustrat look cran could find sign packag call carbonel list jump car cat carbon flavour give also tri set display probabl incorrectli sinc understand go use sy setenv display escapenumb escapenumb escapenumb escapenumb escapenumb r xescapenumb got error messag error xescapenumb xescapenumb modul cannot load sorri type display escapenumb escapenumb escapenumb escapenumb escapenumb r command prompt edit rprofil creat necessari cat nomor rprofil sy putenv display escapenumb escapenumb nomor start xescapenumb server first expect function work termin app open applic util xescapenumb app r xescapenumb discuss mac specif issu belong r

In [ ]:
def pre_process(train) :
  input_ids=[]
  attention_masks=[]
  for i in range(len(train)):
    encoded=tokenizer(train["text"][i], padding="max_length",max_length=512,truncation=True)
    input_ids.append(encoded["input_ids"])
    attention_masks.append(encoded["attention_mask"])
  dataset=pa.DataFrame({
    'input_ids':input_ids,
    'attention_masks':attention_masks,
    'labels':train["lable"]
  })
  return dataset


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
dataset_train=pre_process(train=train)
dataset_test=pre_process(train=test)

In [ ]:
dataset_train.head()

,input_ids,attention_masks,labels
0,"[101, 15068, 12273, 15550, 4605, 20364, 9001, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,"[101, 8814, 2140, 6767, 2497, 2131, 19960, 431...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,"[101, 4012, 18780, 7532, 13229, 4012, 9317, 40...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
3,"[101, 4895, 16402, 2015, 2139, 17603, 6855, 40...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
4,"[101, 4067, 3437, 3124, 2113, 4638, 12667, 603...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0


In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "spam", 1: "notspam"}
label2id = {"spam": 0, "notspam": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset_train.reset_index(drop=True,inplace=True)
dataset_test.reset_index(drop=True,inplace=True)

In [ ]:
from datasets import Dataset
test=Dataset.from_pandas(dataset_test)
train=Dataset.from_pandas(dataset_train)
Dataset={
    'train':train,
    'test':test
}

In [ ]:
Dataset

{'train': Dataset({
     features: ['input_ids', 'attention_masks', 'labels'],
     num_rows: 62349
 }),
 'test': Dataset({
     features: ['input_ids', 'attention_masks', 'labels'],
     num_rows: 21068
 })}

In [ ]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Dataset['train'],
    eval_dataset=Dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.063500,0.045562,0.987896
2,0.022000,0.048835,0.988798


TrainOutput(global_step=7794, training_loss=0.06344047166581089, metrics={'train_runtime': 6537.3725, 'train_samples_per_second': 19.075, 'train_steps_per_second': 1.192, 'total_flos': 1.6518419677605888e+16, 'train_loss': 0.06344047166581089, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.0455617718398571,
 'eval_accuracy': 0.9878963356749573,
 'eval_runtime': 310.6866,
 'eval_samples_per_second': 67.811,
 'eval_steps_per_second': 4.239,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1739032967.489517df9184.13992.3:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1739039819.489517df9184.13992.4:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Chinnolla/my_model/commit/e50630c5e15fb484c77d6ac41ddba62e269446f9', commit_message='End of training', commit_description='', oid='e50630c5e15fb484c77d6ac41ddba62e269446f9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Chinnolla/my_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Chinnolla/my_model'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

text = "perchase the model and get 30% off on other models"
classifier = pipeline(model="chinnolla/my_model")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'spam', 'score': 0.7478772401809692}]

In [ ]:
text="tue escapenumb may escapenumb hadley wickham wrote escapenumb escapenumb escapenumb rolf turner wrote escapenumb escapenumb escapenumb escapenumb escapenumb hadley wickham wrote two possibl solut display escapenumb escapenumb escapenumb escapenumb escapenumb r xescapenumb work without use xterm instal packag carbonel librari carbonel quartz clear life determin frustrat look cran could find sign packag call carbonel list jump car cat carbon flavour give also tri set display probabl incorrectli sinc understand go use sy setenv display escapenumb escapenumb escapenumb escapenumb escapenumb r xescapenumb got error messag error xescapenumb xescapenumb modul cannot load sorri type display escapenumb escapenumb escapenumb escapenumb escapenumb r command prompt edit rprofil creat necessari cat nomor rprofil sy putenv display escapenumb escapenumb nomor start xescapenumb server first expect function work termin app open applic util xescapenumb app r xescapenumb discuss mac specif issu belong r sig mac r help sigsig signatur long core dump r help stat math ethz ch mail list http stat ethz ch mailman listinfo r help pleas read post guid http www r project org post guid html provid comment minim self contain reproduc code"
classifier(text)

[{'label': 'spam', 'score': 0.9998981952667236}]